In [1]:
pip install python-docx pdfplumber

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 3.4 MB/s eta 0:00:02
   ----- ---------------------------------- 0.5/4.0 MB 3.4 MB/s eta 0:00:02
   ------- -------------------------------- 0.8/4.0 MB 1.4 MB/s eta 0:00:03
   ---------- ----------------------------- 1.0/4.0 MB 1.2 MB/s eta 0:00:03
   --------------------- ------------------ 2.1/4.0 MB 1.9 MB/s eta 0:00:01
   -------------------------- ------------- 2.6/4.0 MB 2.2 MB/s eta 0:00:01
   ---------------------------- ----------- 2.9/4.0 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------  3.9/4.0 MB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.8/2.9 MB 5.6 MB/s eta 0:00:01
   ------------------------- ----

In [2]:
pip install transformers torch 

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import nltk
import shutil # Để xóa thư mục

# --- CẤU HÌNH ĐƯỜNG DẪN NLTK DATA TÙY CHỈNH MỚI ---
# Đặt thư mục bạn muốn NLTK lưu trữ dữ liệu của nó.
# Ví dụ: Một thư mục con trong thư mục hiện tại của script, để dễ quản lý.
NLTK_DATA_DIR_NEW = os.path.join(os.getcwd(), "my_new_clean_nltk_data") 

print(f"Thư mục dữ liệu NLTK mới sẽ là: {NLTK_DATA_DIR_NEW}")

# Xóa thư mục cũ nếu nó tồn tại để đảm bảo sạch sẽ hoàn toàn
if os.path.exists(NLTK_DATA_DIR_NEW):
    print(f"Đang xóa thư mục NLTK data cũ tại {NLTK_DATA_DIR_NEW}...")
    shutil.rmtree(NLTK_DATA_DIR_NEW) # Xóa toàn bộ cây thư mục
    print("Đã xóa.")

# Tạo thư mục mới
os.makedirs(NLTK_DATA_DIR_NEW, exist_ok=True)
print(f"Đã tạo thư mục mới cho NLTK data tại {NLTK_DATA_DIR_NEW}")

# Thêm đường dẫn tùy chỉnh mới vào danh sách tìm kiếm của NLTK.
# Quan trọng: Đảm bảo nó được thêm VÀO ĐẦU danh sách để ưu tiên.
if NLTK_DATA_DIR_NEW not in nltk.data.path:
    nltk.data.path.insert(0, NLTK_DATA_DIR_NEW) 

# --- Đảm bảo gói 'punkt' được tải xuống một lần duy nhất vào thư mục mới ---
print("\nĐang tiến hành tải xuống gói 'punkt' vào thư mục mới...")
try:
    nltk.download('punkt', download_dir=NLTK_DATA_DIR_NEW)
    print(f"NLTK 'punkt' gói đã được tải xuống thành công vào {NLTK_DATA_DIR_NEW}.")
except Exception as e:
    print(f"Lỗi khi tải xuống NLTK 'punkt' gói (có thể là do mạng hoặc quyền): {e}")
    print("WARNING: NLTK 'punkt' could not be downloaded successfully. Sentence tokenization might fall back to a simpler method.")
    # Có thể thêm raise SystemExit(e) ở đây nếu bạn muốn dừng hoàn toàn script khi tải thất bại.

print("\nCấu hình NLTK hoàn tất. Bạn có thể chạy cell tiếp theo với code chunking.")

# Sau khi chạy cell này thành công, nltk.sent_tokenize() sẽ tự động hoạt động.

Thư mục dữ liệu NLTK mới sẽ là: c:\Users\VuongQuan14\Documents\Zalo Received Files\my_new_clean_nltk_data
Đã tạo thư mục mới cho NLTK data tại c:\Users\VuongQuan14\Documents\Zalo Received Files\my_new_clean_nltk_data

Đang tiến hành tải xuống gói 'punkt' vào thư mục mới...


[nltk_data] Downloading package punkt to
[nltk_data]     c:\Users\VuongQuan14\Documents\Zalo Received
[nltk_data]     Files\my_new_clean_nltk_data...


NLTK 'punkt' gói đã được tải xuống thành công vào c:\Users\VuongQuan14\Documents\Zalo Received Files\my_new_clean_nltk_data.

Cấu hình NLTK hoàn tất. Bạn có thể chạy cell tiếp theo với code chunking.


[nltk_data]   Unzipping tokenizers\punkt.zip.


In [9]:
import re
import os
# Sửa đổi import ở đây
import docx # Import cả gói docx
from docx import Document # Vẫn import Document class như cũ
import nltk
import json

# Hàm phân tách câu dựa trên Regex đơn giản
def simple_sent_tokenize(text):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-ZĐ][a-zđàáạãảăằắặẵẳâầấậẫẩèéẹẽẻêềếệễểìíịĩỉòóọõỏôồốộỗổơờớợỡởùúụũủưừứựữửỳýỵỹỷ]\.)(?<=\.|\?|\!)\s+(?=[A-ZĐ])', text)
    return [s.strip() for s in sentences if s.strip()]

# Hàm hỗ trợ phân tách sub-chunk dựa trên số lượng câu
def _sentence_based_sub_chunking(text, max_sentences, base_metadata, sources, chunks_list_ref):
    sentences = simple_sent_tokenize(text)
    current_sub_chunk_sentences = []

    for sentence in sentences:
        current_sub_chunk_sentences.append(sentence)
        if len(current_sub_chunk_sentences) >= max_sentences:
            chunks_list_ref.append({
                "content": " ".join(current_sub_chunk_sentences),
                "metadata": base_metadata,
                "sources": sorted(list(set(sources)))
            })
            current_sub_chunk_sentences = []
    
    if current_sub_chunk_sentences:
        chunks_list_ref.append({
            "content": " ".join(current_sub_chunk_sentences),
            "metadata": base_metadata,
            "sources": sorted(list(set(sources)))
        })


def read_docx_content_and_tables(filepath):
    """
    Reads a .docx file and extracts content from paragraphs and tables in order.
    Returns:
        tuple: (list of content strings, list of corresponding source line numbers)
    """
    try:
        doc = Document(filepath)
        all_flat_lines = [] # Stores (content_string, simple_line_number) tuples
        current_simple_line_num = 1 # Sequential line number for all extracted text

        # Iterate through elements in the body to maintain order
        for block in doc.element.body: # doc.element.body contains _element objects
            # Check if it's a paragraph element
            if block.tag.endswith('p'):
                # Get the paragraph object from the element
                para = docx.text.paragraph.Paragraph(block, doc) # Sử dụng docx.text.paragraph
                stripped_text = para.text.strip()
                if stripped_text:
                    all_flat_lines.append((stripped_text, current_simple_line_num))
                    current_simple_line_num += 1
            # Check if it's a table element
            elif block.tag.endswith('tbl'):
                # Get the table object from the element
                table = docx.table.Table(block, doc) # Sử dụng docx.table
                for row in table.rows:
                    for cell in row.cells:
                        cell_text = cell.text.strip()
                        if cell_text:
                            all_flat_lines.append((cell_text, current_simple_line_num))
                            current_simple_line_num += 1
        
        # Filter empty lines and separate content from sources
        final_lines_with_sources = [(text, src) for text, src in all_flat_lines if text]
        
        lines_only = [item[0] for item in final_lines_with_sources]
        sources_only = [item[1] for item in final_lines_with_sources]

        return lines_only, sources_only

    except Exception as e:
        print(f"Lỗi khi đọc file DOCX {filepath}: {e}")
        return [], []

def read_txt_lines_with_sources(filepath):
    """
    Reads a .txt file and returns its content as a list of cleaned lines with source line numbers.
    """
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            lines = []
            for i, line in enumerate(f):
                stripped_line = line.strip()
                if stripped_line:
                    lines.append((stripped_line, i + 1))
        return [item[0] for item in lines], [item[1] for item in lines] # Separate lines and sources
    except Exception as e:
        print(f"Lỗi khi đọc file TXT {filepath}: {e}")
        return [], []


def chunk_document_from_file(filepath, max_sentences_per_sub_chunk=None, min_chunk_char_size=50):
    lines = []
    line_sources = [] # Store original line numbers
    
    if filepath.endswith('.docx'):
        lines, line_sources = read_docx_content_and_tables(filepath) 
    elif filepath.endswith('.txt'):
        lines, line_sources = read_txt_lines_with_sources(filepath)
    else:
        print(f"Loại file không được hỗ trợ cho {filepath}. Chỉ hỗ trợ .docx và .txt.")
        return []

    if not lines:
        print("Không có dòng nào được đọc từ file hoặc file trống.")
        return []

    # --- DEBUG: Toàn bộ danh sách 'lines' sau khi đọc từ file (bao gồm bảng): ---
    print(f"\n--- DEBUG: Toàn bộ danh sách 'lines' sau khi đọc từ file (bao gồm bảng): ---")
    for i, line_content in enumerate(lines):
        # Kiểm tra nếu line_sources[i] tồn tại
        if i < len(line_sources):
            print(f"Line {line_sources[i]}: {repr(line_content)}") 
        else:
            print(f"Line ???: {repr(line_content)}") # Fallback if source index is out of bounds
    print("----------------------------------------------------------\n")
    # ----------------------------------------------------------------------

    chunks = []
    current_chapter = None
    current_section = None
    current_article = None
    current_clause = None
    current_point = None
    current_content_lines = []
    current_sources = []
    
    circular_id = None

    patterns = {
        "chapter": re.compile(r"Chương ([IVXLCDM]+)", re.IGNORECASE),
        "section": re.compile(r"Mục (\d+)\.?"),
        "article": re.compile(r"Điều (\d+)\.?\s*(.*)"),
        "clause": re.compile(r"(\d+)\.\s*(.*)"),
        "point": re.compile(r"([a-zđ])\)\s*(.*)"),
        "legal_basis": re.compile(r"^Căn cứ .+;?$"), 
        "initial_header_start": re.compile(r"^(?:BỘ NÔNG NGHIỆP VÀ MÔI TRƯỜNG|THÔNG TƯ|QUY ĐỊNH|NGHỊ ĐỊNH)", re.IGNORECASE),
        "initial_header_end_marker": re.compile(r"^của Bộ Nông nghiệp và Môi trường$"), 
        "final_intro_line_start": re.compile(r"^Theo đề nghị của Vụ trưởng Vụ Khoa học và Công nghệ;"),
        "chapter_start_marker": re.compile(r"^Chương I"), 
        
        "circular_id": re.compile(r"Số:\s*(?:[^/\s]*\s*)?([0-9A-ZĐ]+/[0-9]{4}/[A-ZĐ-]+)", re.IGNORECASE) 
    }

    def _save_chunk_internal(content_str, metadata, sources):
        if content_str:
            chunks.append({
                "content": content_str,
                "metadata": metadata,
                "sources": sorted(list(set(sources)))
            })

    def save_current_structural_chunk_with_merge():
        nonlocal current_chapter, current_section, current_article, current_clause, current_point
        nonlocal current_content_lines, current_sources, circular_id

        if current_content_lines:
            full_content_str = " ".join(current_content_lines).strip()
            
            base_metadata = {
                "circular_id": circular_id, 
                "chapter": current_chapter,
                "section": current_section,
                "article": current_article,
                "clause": current_clause,
                "point": current_point
            }
            
            if len(full_content_str) < min_chunk_char_size and not patterns["chapter"].match(full_content_str):
                return 
            
            if max_sentences_per_sub_chunk is not None:
                initial_sentences = simple_sent_tokenize(full_content_str)
                if len(initial_sentences) > max_sentences_per_sub_chunk:
                    _sentence_based_sub_chunking(full_content_str, max_sentences_per_sub_chunk, base_metadata, current_sources, chunks)
                else:
                    _save_chunk_internal(full_content_str, base_metadata, current_sources)
            else:
                _save_chunk_internal(full_content_str, base_metadata, current_sources)
            
            current_content_lines.clear()
            current_sources.clear()

    # --- Bắt đầu quá trình duyệt dòng ---
    # Logic xác định khối header ban đầu, bao gồm tìm circular_id
    
    # Tìm circular_id trong toàn bộ danh sách `lines` đã đọc
    for k in range(len(lines)):
        content_from_line = lines[k]
        circular_id_match = patterns["circular_id"].search(content_from_line)
        if circular_id_match:
            circular_id = circular_id_match.group(1).strip()
            break 

    header_block_content_lines = []
    header_block_sources = []
    
    i = 0
    while i < len(lines):
        raw_line = lines[i]
        line_num = line_sources[i]
        content = raw_line.strip()

        if not content:
            i += 1
            continue
        
        if patterns["legal_basis"].search(content) or patterns["final_intro_line_start"].search(content):
            break

        if patterns["initial_header_end_marker"].search(content):
            header_block_content_lines.append(content)
            header_block_sources.append(line_num)
            i += 1
            break
        
        if patterns["chapter_start_marker"].match(content):
            break 
            
        header_block_content_lines.append(content)
        header_block_sources.append(line_num)
        i += 1

    if header_block_content_lines:
        _save_chunk_internal(" ".join(header_block_content_lines).strip(), 
                             {'circular_id': circular_id, 'chapter': None, 'section': None, 'article': None, 'clause': None, 'point': None, 'type': 'initial_header'}, 
                             header_block_sources)
    
    # --- PHASE 2: Xử lý các dòng "Căn cứ" ---
    while i < len(lines):
        raw_line = lines[i]
        line_num = line_sources[i]
        content = raw_line.strip()

        if not content:
            i += 1
            continue
        
        if patterns["final_intro_line_start"].search(content) or patterns["chapter_start_marker"].match(content):
            break

        if patterns["legal_basis"].search(content):
            _save_chunk_internal(content, 
                                 {'circular_id': circular_id, 'chapter': None, 'section': None, 'article': None, 'clause': None, 'point': None, 'type': 'legal_basis'}, 
                                 [line_num])
        i += 1
    
    # --- PHASE 3: Xử lý dòng "Theo đề nghị..." cuối phần mở đầu ---
    while i < len(lines):
        raw_line = lines[i]
        line_num = line_sources[i]
        content = raw_line.strip()

        if not content:
            i += 1
            continue
        
        if patterns["final_intro_line_start"].search(content):
            intro_final_content = content
            intro_final_sources = [line_num]
            
            j = i + 1
            while j < len(lines):
                next_line_content = lines[j].strip()
                if not next_line_content: 
                    j += 1
                    continue
                if patterns["chapter_start_marker"].match(next_line_content):
                    break 
                intro_final_content += " " + next_line_content
                intro_final_sources.append(line_sources[j])
                j += 1
            _save_chunk_internal(intro_final_content, 
                                 {'circular_id': circular_id, 'chapter': None, 'section': None, 'article': None, 'clause': None, 'point': None, 'type': 'final_intro'}, 
                                 intro_final_sources)
            i = j 
            break 
        else:
            break 

    # --- PHASE 4: Xử lý phần cấu trúc chính (Chương, Điều, Khoản, Điểm) ---
    while i < len(lines):
        raw_line = lines[i]
        line_num = line_sources[i]
        content_without_source = raw_line.strip()
        current_sources_for_line = [line_num]

        if not content_without_source:
            i += 1
            continue

        is_structural_change = False
        if patterns["chapter"].match(content_without_source):
            save_current_structural_chunk_with_merge()
            current_chapter = patterns["chapter"].match(content_without_source).group(0).strip()
            current_section = None; current_article = None; current_clause = None; current_point = None
            is_structural_change = True
        elif patterns["section"].match(content_without_source):
            save_current_structural_chunk_with_merge()
            current_section = patterns["section"].match(content_without_source).group(0).strip()
            current_article = None; current_clause = None; current_point = None
            is_structural_change = True
        elif patterns["article"].match(content_without_source):
            save_current_structural_chunk_with_merge()
            current_article = patterns["article"].match(content_without_source).group(0).strip()
            current_clause = None; current_point = None
            is_structural_change = True
        elif patterns["clause"].match(content_without_source) and current_article:
            save_current_structural_chunk_with_merge()
            current_clause = patterns["clause"].match(content_without_source).group(0).strip()
            current_point = None
            is_structural_change = True
        elif patterns["point"].match(content_without_source) and current_clause:
            save_current_structural_chunk_with_merge()
            current_point = patterns["point"].match(content_without_source).group(0).strip()
            is_structural_change = True
        
        if is_structural_change:
            current_content_lines.append(content_without_source)
            current_sources.extend(current_sources_for_line)
        else:
            current_content_lines.append(content_without_source)
            current_sources.extend(current_sources_for_line)
        
        i += 1 
        
    save_current_structural_chunk_with_merge()

    return chunks

# --- CÁCH SỬ DỤNG VỚI ĐƯỜNG DẪN FILE THÔNG TƯ ---

file_path = "C:/Users/VuongQuan14/Downloads/THÔNG TƯ.docx"
output_json_file = "D:/Project/RAG_NLP_build_-main/Data/chunked_document.json" 

max_sentences_per_sub_chunking = 3 
min_chunk_char_size_to_merge = 50 

chunked_data = chunk_document_from_file(
    file_path, 
    max_sentences_per_sub_chunk=max_sentences_per_sub_chunking,
    min_chunk_char_size=min_chunk_char_size_to_merge
)

# --- IN KẾT QUẢ VỚI METADATA XUỐNG DÒNG ---
print(f"Tổng số chunk được tạo: {len(chunked_data)}")
print("n--- Toàn bộ các chunk từ file thông tư (có xử lý intro và gộp chunk nhỏ) ---")
for i, chunk in enumerate(chunked_data):
    display_content = chunk['content']
    if len(display_content) > 300: 
        display_content = display_content[:297] + "..."
    
    print(f"\n--- Chunk {i+1} ---")
    print(f"Nội dung: {display_content}")
    print("Metadata:")
    for key, value in chunk['metadata'].items():
        print(f"  {key}: {value}")
    print(f"Sources (Line Nums): {chunk['sources']}")
    print("-" * 30)

# --- XUẤT CHUNK RA FILE JSON ---
try:
    with open(output_json_file, 'w', encoding='utf-8') as f:
        json.dump(chunked_data, f, ensure_ascii=False, indent=4)
    print(f"\nĐã xuất thành công {len(chunked_data)} chunk ra file: {output_json_file}")
except Exception as e:
    print(f"Lỗi khi xuất file JSON: {e}")


--- DEBUG: Toàn bộ danh sách 'lines' sau khi đọc từ file (bao gồm bảng): ---
Line 1: 'THÔNG TƯ'
Line 2: 'Name'
Line 3: 'Căn cứ Luật Khoa học và Công nghệ ngày 18 tháng 6 năm 2013, được sửa đổi, bổ sung bởi Luật sửa đổi, bổ sung một số điều của 11 luật có liên quan đến quy hoạch ngày 15 tháng 6 năm 2018 và Luật sửa đổi, bổ sung một số điều của Luật Sở hữu trí tuệ ngày 16 tháng 6 năm 2022;'
Line 4: 'Căn cứ Nghị quyết số 193/2025/QH15 ngày 19 tháng 02 năm 2025 của Quốc hội về thí điểm một số cơ chế, chính sách đặc biệt tạo đột phá phát triển khoa học, công nghệ, đổi mới sáng tạo và chuyển đổi số quốc gia;'
Line 5: 'Căn cứ Nghị định số 08/2014/NĐ-CP ngày 27 tháng 01 năm 2014 của Chính phủ quy định chi tiết và hướng dẫn thi hành một số điều của Luật khoa học và công nghệ;'
Line 6: 'Căn cứ Nghị định số 11/2014/NĐ-CP ngày 18 tháng 02 năm 2014 của Chính phủ ban hành về hoạt đông thông tin khoa học và công nghệ;'
Line 7: 'Căn cứ Nghị định số 42/2022/NĐ-CP ngày 24 tháng 6 năm 2022 của Chính phủ